In [6]:
import docker, os

a = %pwd
wd = (a.rsplit('/',2))[0]

GENE = "ITGAL"
BLAST_Results_ncbi=os.path.join("homologue", GENE, "NM_001114380_ITGAL.fasta")
BLAST_Results_uniprot=os.path.join("homologue", GENE, "B20210131A94466D2655679D1FD8953E075198DA8030E49B.fasta")
REF=os.path.join("reference", GENE, "ITGAL_prot.fasta")
D_MEME='quay.io/biocontainers/meme:5.3.0--py37pl526h072abfd_0'

WORK_HOST_DIRECTORY=wd
WORK_CONT_DIRECTORY='/opt/project'

In [7]:
client = docker.from_env()
bwa_docker_cont = client.containers.run(
    image=D_MEME,
    volumes={WORK_HOST_DIRECTORY: {'bind': WORK_CONT_DIRECTORY,'mode': 'rw'}},
    working_dir= WORK_CONT_DIRECTORY+'/data/motif/',
    entrypoint="bash",
    command='{0}/scripts_bash/meme.sh {1} {2} {3} {4}'.format(WORK_CONT_DIRECTORY, BLAST_Results_ncbi, BLAST_Results_uniprot, REF, GENE))

In [ ]:
from Bio import motifs

with open(os.path.join(wd, "data/motif", GENE, "meme/meme.xml")) as file:
    meme_record = motifs.parse(file, "MEME")

In [ ]:
from scripts_python.vcf_gff import gff2rec

motif_in_ref = gff2rec(os.path.join(wd,"data/motif", GENE, "fimo/fimo.gff"))
for seq in motif_in_ref:
    for feature in seq.features:
        print(feature)

In [ ]:
from Bio import motifs

with open(os.path.join(wd, "data/motif", GENE, "mast/mast.xml")) as f:
    mast_records = motifs.parse(f, "MAST")

for motif in mast_records:
    print(motif.num_occurrences)